### Eigenestimator Class

In [ ]:
%run eigenfunction_estimator_decomp_class.ipynb
from scipy.linalg import lstsq
class Tx_estimator:

    def __init__(self, basis_functions, data_real, data_imag, data_hx, deltat, real_eig, im_eig, N, M, lambda_values):
        # self.basis_functions: basis_functions used for eigenestimator
        # self.data_real: data for estimation of eigenfunctions for real negative eigenvalues
        # self.data_imag: data for estimation of eigenfunctions for imaginary eigenvalues
        # self.deltat: time different between each consecutive state in the data
        # self.im_eig: imaginary eigenvalue
        # self.real_eig: real eigenvalue
        # self.N: number of imaginary eigenvalues 
        # self.M: number of real eigenvalues
        # self.lambda_values: chosen lambda values for T(x) construction
        # self.hx_data: data for estimation of h(x)
        self.basis_functions = basis_functions
        self.data_real = data_real
        self.data_imag = data_imag
        self.deltat = deltat
        self.im_eig = im_eig
        self.real_eig = real_eig
        self.N = N
        self.M = M
        self.lambda_values = lambda_values
        self.data_hx = data_hx

    def Tx_estimate(self):
        # GENERATE EIGENFUNCTIONS FOR MULTIPLES OF REAL EIGENVALUE
        self.real_array = {}
        for m in range(1, self.M+1):
            # Estimate original eigenfunction
            self.system_real = eigenestimator_decomp(self.basis_functions, self.data_real, self.deltat, self.real_eig*m, np.array([[1, 4]]), np.array([[0.75, 2]]))
            self.system_real.estimate()
            print('Eigenfunction for m = %i, error = %f' %(m, self.system_real.error))
            self.real_array[m] = self.system_real.estimated_function

        # Assemble
        self.eigen_functions_real = lambda x: np.concatenate((np.array([1.]), np.array([self.real_array[m](x) for m in range(1, self.M+1)])))
        
        # GENERATE EIGENFUNCTIONS FOR NEGATIVE MULTIPLES OF REAL EIGENVALUE
        self.im_array = {}
        for n in range(-self.N, 0):
            self.system_im = eigenestimator_decomp(self.basis_functions, self.data_imag, self.deltat, self.im_eig*n, np.array([[1, 2]]), np.array([[3.75, 4]]))
            self.system_im.estimate()
            print('Eigenfunction for n = %i, error = %f' %(n, self.system_im.error))
            self.im_array[n] = self.system_im.estimated_function

        # Assemble
        self.eigen_functions_im = lambda x: np.concatenate((
            np.array([self.im_array[n](x) for n in range(-self.N, 0)]),
            np.array([1.]),
            np.conj(np.flip(np.array([self.im_array[n](x) for n in range(-self.N, 0)])))
        ))

        # MULTIPLY THE REAL AND IMAGINARY EIGENVALUED EIGENFUNCTIONS
        self.eigen_functions = lambda x: np.outer(self.eigen_functions_real(x), self.eigen_functions_im(x)).flatten()
        self.eigen_values = np.array([[self.real_eig*m + self.im_eig*n for n in range(-self.N, self.N+1)] for m in range(0, self.M+1)]).flatten()

        self.now = np.array([self.data_real[i][:-1] for i in range(len(self.data_real))])
        row1, column1, depth1 = self.now.shape
        self.now = self.now.reshape(row1*column1, depth1)
        
        self.later = np.array([self.data_real[i][1:] for i in range(len(self.data_real))])
        row2, column2, depth2 = self.later.shape
        self.later = self.later.reshape(row2*column2, depth2)

        self.hx_values = self.now[:, 1]

        varphi_y = np.array([self.eigen_functions(y) for y in self.later])
        varphi_x = np.array([self.eigen_functions(x) for x in self.now])

        self.coefficients = []
        for i in range(len(self.lambda_values)):
            self.A = (varphi_y - (1 + self.lambda_values[i]*self.deltat)*varphi_x) / self.deltat
            self.b, _, _, _ = lstsq(self.A, self.hx_values)
            self.rmse = np.sqrt(np.mean(np.abs(self.hx_values - self.A @ self.b)**2))
            print(f'RMSE of Least Squares for T{i+1}(x): ', self.rmse)
            self.coefficients.append(self.b)
        
        self.Tx = lambda x: np.array([np.dot(self.eigen_functions(x), coeff) for coeff in self.coefficients])

    def plot(self):
        fig, ax = plt.subplots(2)
        # Create meshgrid
        x, y = np.linspace(0.1, 6.0, 60), np.linspace(0.1, 6.0, 60)
        X, Y = np.meshgrid(x, y)
        Z1, Z2 = np.zeros((len(x), len(y)), dtype=complex), np.zeros((len(x), len(y)), dtype=complex)
        for i in range(len(x)):
            for j in range(len(y)):
                val = self.Tx(np.array([x[i], y[j]]))
                Z1[i, j], Z2[i, j] = val[0], val[1]
        no_data = ((X - 1)**2 + (Y - 3)**2 <= 0.1) | (X<=0.2) | (Y<=0.1) | (X+Y>=7) | (X+Y<=2)
        Z1[no_data] = np.nan
        Z2[no_data] = np.nan
        Zmax = min(20.0, max(np.nanmax(np.abs(Z1)), np.nanmax(np.abs(Z2))))
        for k in range(0, 2): 
            levels = np.linspace(-Zmax, Zmax, 21)
            cmap = plt.get_cmap('coolwarm')
            norm = mcolors.BoundaryNorm(levels, cmap.N)
            contour = ax[k].pcolormesh(X, Y, (np.real(Z1) if k == 0 else np.real(Z2)), cmap='coolwarm', norm=norm)
            ax[k].set_xlabel(r'$x_1$'), ax[k].set_ylabel(r'$x_2$'), ax[k].set_title(r'$T_{%d}(x)$' % (k+1))
            fig.colorbar(contour)
        plt.tight_layout(), plt.show()
    
    def jacobian(self, x):
        J = np.zeros((len(x), len(x)), dtype = complex)
        for i in range(len(x)):
            h = np.zeros(len(x))
            h[i] = 1.0
            J[:, i] = (self.Tx(x + h*1e-6) - self.Tx(x)) / 1e-6
        return J